# LlamaStack Workshop - Client Demo

This notebook demonstrates how to interact with your deployed LlamaStack distribution programmatically.

## Prerequisites
- You have deployed a model (Llama 3.2-3B) in your project
- You have enabled the GenAI Playground (which creates a LlamaStack Distribution)

## What You'll Learn
1. How to list available models
2. How to list available tools (MCP servers)
3. How to make chat completions
4. How to use tool calling

In [ ]:
# Install required packages
%pip install -q requests

In [ ]:
import requests
import json
import os

# Configuration - UPDATE THIS with your project name!
PROJECT_NAME = "user-XX"  # <-- Change XX to your user number (e.g., user-05)

# LlamaStack endpoint (internal OpenShift service)
LLAMASTACK_URL = f"http://lsd-genai-playground-service.{PROJECT_NAME}.svc.cluster.local:8321"

print(f"Project: {PROJECT_NAME}")
print(f"LlamaStack URL: {LLAMASTACK_URL}")

## 1. List Available Models

Let's see what models are available in your LlamaStack distribution.

In [ ]:
response = requests.get(f"{LLAMASTACK_URL}/v1/models", timeout=10)
models = response.json().get("data", [])

# Filter to LLM models only
llm_models = [m for m in models if m.get("model_type") == "llm"]

print(f"🤖 LLM Models Available: {len(llm_models)}")
print("=" * 50)
for m in llm_models:
    print(f"  • {m.get('identifier')} ({m.get('provider_id')})")
    
# Store the first model ID for later use
# Note: identifier already includes the provider prefix (e.g., "vllm-inference-1/llama-32-3b-instruct")
if llm_models:
    MODEL_ID = llm_models[0].get('identifier')
    print(f"\n📌 Using model: {MODEL_ID}")

## 2. List Available Tools (MCP Servers)

Tools are provided by MCP servers. Let's see what's available.

In [ ]:
response = requests.get(f"{LLAMASTACK_URL}/v1/tools", timeout=10)
data = response.json()
tools = data if isinstance(data, list) else data.get("data", [])

# Group by toolgroup (MCP server)
toolgroups = {}
for t in tools:
    tg = t.get("toolgroup_id", "unknown")
    if tg not in toolgroups:
        toolgroups[tg] = []
    toolgroups[tg].append(t.get("name", "unknown"))

# Count MCP servers (exclude builtin)
mcp_servers = [tg for tg in toolgroups.keys() if tg.startswith("mcp::")]

print(f"🛠️ MCP Servers: {len(mcp_servers)}")
print(f"📊 Total Tools: {len(tools)}")
print("=" * 50)
for tg, tool_list in sorted(toolgroups.items()):
    icon = "🌤️" if "weather" in tg else "🔧"
    print(f"\n{icon} {tg} ({len(tool_list)} tools)")
    for tool in tool_list:
        print(f"   • {tool}")

## 3. Simple Chat Completion (No Tools)

Let's test a basic chat completion first (without using any tools).

In [ ]:
payload = {
    "model": MODEL_ID,
    "messages": [
        {"role": "user", "content": "What is the capital of France? Answer in one sentence."}
    ],
    "temperature": 0.7,
    "max_tokens": 256
}

print(f"🤖 Using model: {MODEL_ID}")
print("=" * 50)

response = requests.post(
    f"{LLAMASTACK_URL}/v1/openai/v1/chat/completions",
    json=payload,
    timeout=60
)

if response.status_code == 200:
    result = response.json()
    content = result.get("choices", [{}])[0].get("message", {}).get("content", "")
    print(f"\n📝 Response:")
    print(content)
else:
    print(f"❌ Error: {response.status_code} - {response.text}")

## 4. Test Weather MCP Tool

Now let's test the Weather MCP tool! This query will make the AI use the weather tool to get real data.

In [ ]:
# Test Weather MCP - List all weather stations
weather_question = "List all available weather stations"

payload = {
    "model": MODEL_ID,
    "messages": [
        {"role": "user", "content": weather_question}
    ],
    "temperature": 0.1,  # Lower temperature for more consistent tool usage
    "max_tokens": 1024
}

print(f"🌤️ Weather Query: {weather_question}")
print("=" * 50)

response = requests.post(
    f"{LLAMASTACK_URL}/v1/openai/v1/chat/completions",
    json=payload,
    timeout=120  # Longer timeout for tool calls
)

if response.status_code == 200:
    result = response.json()
    content = result.get("choices", [{}])[0].get("message", {}).get("content", "")
    print(f"\n📝 Response:")
    print(content)
else:
    print(f"❌ Error: {response.status_code} - {response.text}")

## 5. Explore on Your Own!

Try modifying the query below to test different weather questions:
- "Get weather statistics"
- "Search for weather in Tokyo"
- "Get current weather for station VIDP"

Station codes: VIDP = New Delhi, RJTT = Tokyo, KJFK = New York, EGLL = London, YSSY = Sydney

In [ ]:
# Your turn! Modify this query to try different weather questions
my_question = "Get weather statistics"

payload = {
    "model": MODEL_ID,
    "messages": [
        {"role": "user", "content": my_question}
    ],
    "temperature": 0.1,
    "max_tokens": 1024
}

print(f"🔍 Query: {my_question}")
print("=" * 50)

response = requests.post(
    f"{LLAMASTACK_URL}/v1/openai/v1/chat/completions",
    json=payload,
    timeout=120
)

if response.status_code == 200:
    result = response.json()
    content = result.get("choices", [{}])[0].get("message", {}).get("content", "")
    print(f"\n📝 Response:")
    print(content)
else:
    print(f"❌ Error: {response.status_code} - {response.text}")